In [ ]:
import openai

# Create OpenAI client for OpenRouter
client = openai.OpenAI(
    api_key="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",  # Replace with your actual OpenRouter API key
    base_url="https://openrouter.ai/api/v1"
)

# Make a request
response = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct",  # Try a different model if GPT-4 isn't working
    messages=[{"role": "user", "content": "Hello!"}]
)

# Print the response
print(response.choices[0].message.content)


 Hello there! How can I help you today? Feel free to ask me anything you'd like to know, I'll do my best to assist you. If you have a specific question or topic in mind, that would be great. If not, I can provide information on a wide range of subjects, such as technology, science, art, history, literature, and more. Let's get started! What would you like to know?


Chatbot


In [ ]:
import openai

# Create OpenAI client for OpenRouter
client = openai.OpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1"
)

# Chat loop
print("Chatbot ready! Type 'exit' to stop.")
conversation = []

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    conversation.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="mistralai/mistral-7b-instruct",
        messages=conversation
    )

    ai_reply = response.choices[0].message.content
    print("AI:", ai_reply)

    conversation.append({"role": "assistant", "content": ai_reply})


Chatbot ready! Type 'exit' to stop.
You: hello
AI:  Hello there! How may I assist you today? If you have any questions or need help with something, feel free to ask. I'm here to help! If you just want to chat, that's great too! What's new with you?
You: can you tell small story
AI:  Of course! Here's a short story for you:

Once upon a time, in a small village nestled between the mountains and the sea, there lived a kind and hardworking farmer named Tom. Tom spent his days tending to his fields, working tirelessly to ensure a good harvest for himself and his family.

One day, as Tom was working in the field, he noticed a tiny bird that had fallen from its nest. The bird was injured and couldn't fly. Tom's heart went out to the poor creature and he carefully picked it up and brought it home.

He made a little nest for the bird in a box and fed it tiny pieces of bread soaked in milk. Days went by, and the bird slowly regained its strength. Tom grew very fond of the little bird and named 

News Summarizer

In [ ]:
pip install langchain openai requests beautifulsoup4


In [ ]:
pip install --upgrade langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:

import openai
import requests
from bs4 import BeautifulSoup

# Set up OpenRouter API
api_key = "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"
base_url = "https://openrouter.ai/api/v1"

# Fetch news article from a website
def fetch_news(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        article_text = " ".join([p.get_text() for p in paragraphs])
        return article_text[:4000]  # Truncate to fit token limits
    return None

# Summarization function using OpenRouter API directly
def summarize_news(article_text):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistralai/mistral-7b-instruct",  # Try "gpt-4" for better summaries
        "messages": [{
            "role": "user",
            "content": f"Summarize this news article in **5 sentences** with key details:\n\n{article_text}"
        }]
    }

    response = requests.post(f"{base_url}/chat/completions", headers=headers, json=data)

    if response.status_code == 200:
        summary = response.json()["choices"][0]["message"]["content"]
        # Format the summary into separate lines
        return "\n".join(summary.split(". "))
    else:
        return f"Error: {response.json()}"

# Example usage
news_url = "https://www.thehindu.com"
article = fetch_news(news_url)

if article:
    summary = summarize_news(article)
    print("\n=== News Summary ===\n", summary)
else:
    print("Failed to fetch news article.")




=== News Summary ===
  This edition of The Hindu includes a variety of sections:

1
Books of the week, reviews, new titles, and features are presented.
2
The section 'Data Point' offers fact-based analysis on headlines.
3
'First Day First Show' provides news and reviews from the cinema and streaming world.
4
In 'Health Matters', Ramya Kannan offers insights on maintaining good health.
5
'The View From India' provides an Indian perspective on world affairs
'Science For All' simplifies science news and adds fun
'Karnataka Today' and 'Today's Cache' offer news highlights from Karnataka and the top 5 technology stories of the day, respectively
The paper is published by THG Publishing PVT LTD
and is copyrighted.


Resume Analyser

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.8 MB/s eta 0:00:00


In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF for PDFs
from docx import Document  # For Word files
import os
import requests

# OpenRouter API setup
api_key = "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"
base_url = "https://openrouter.ai/api/v1"

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

# Function to extract text from a Word document
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

# Function to analyze resume using OpenRouter AI
def analyze_resume(resume_text):
    # Limit text to prevent exceeding model token limits
    max_tokens = 4000
    truncated_text = resume_text[:max_tokens]

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    prompt = f"""
    Analyze the following resume and provide:
    1. Summary of candidate details (Name, Skills, Experience, Education)
    2. Strengths of the resume
    3. Areas for improvement (e.g., missing details, formatting issues)
    4. Suggestions for better job fit

    Resume:
    {truncated_text}
    """

    data = {
        "model": "mistralai/mistral-7b-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(f"{base_url}/chat/completions", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.json()}"

# Main function to process resume
def process_resume(file_path):
    if not os.path.exists(file_path):
        return "Error: File not found."

    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".pdf":
        resume_text = extract_text_from_pdf(file_path)
    elif ext == ".docx":
        resume_text = extract_text_from_docx(file_path)
    else:
        return "Unsupported file format. Please upload a PDF or DOCX."

    print("\nAnalyzing Resume...\n")
    analysis = analyze_resume(resume_text)
    print("\n=== Resume Analysis ===\n", analysis)

# Interactive file selection
if __name__ == "__main__":
    file_path = input("Enter the full path of the resume file (PDF or DOCX): ").strip()
    process_resume(file_path)


Enter the full path of the resume file (PDF or DOCX): /content/resume_sample.pdf

Analyzing Resume...


=== Resume Analysis ===
  1. Summary of candidate details:
   - Name: Hayden Smith
   - Address: 214 Mitre Avenue, Park Hill, 3045
   - Mobile Number: 04501 123 456
   - Email: haydensmith@email.com
   - Career Objective: Seeking casual or part-time customer service work in a sports retail environment
   - Availability: Monday – Friday: 4.30pm – 10.00pm, Saturday – Sunday: 8.00am – 11.00pm, up to 20 hours per week
   - Key Skills: Customer service, numeracy, communication, teamwork, organization, problem-solving, soccer-related skills
   - Education: Year 11 at Park Hill Secondary College, taking Maths, English, Business Management, VET studies in Sport and Recreation
   - Work Experience: Customer service volunteer at Park Hill Soccer Club Canteen (Dec 2016 – Mar 2017), Newspaper deliverer at Argo Newsagency
   - Leadership Roles: Soccer umpire for under 14 team at Park Hill Soccer 

AI-Based Medical Diagnosis Assisstent


In [ ]:
import requests

# OpenRouter API setup
api_key = "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"
base_url = "https://openrouter.ai/api/v1"

def get_medical_diagnosis(symptoms):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    prompt = f"""
    You are an AI medical assistant. Analyze the following symptoms and provide:
    1. Possible medical conditions
    2. Recommended actions (e.g., rest, consult a doctor, emergency care)
    3. Precautionary measures

    Symptoms: {symptoms}
    """

    data = {
        "model": "mistralai/mistral-7b-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(f"{base_url}/chat/completions", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.json()}"

# Example usage
user_symptoms = input("Enter your symptoms: ")
diagnosis = get_medical_diagnosis(user_symptoms)
print("\n=== AI Diagnosis ===\n", diagnosis)


Enter your symptoms: cold

=== AI Diagnosis ===
  1. Possible medical conditions: The symptoms you've described (cold) are common and can be caused by various viruses, the most common of which is the common cold. Other possible causes could be influenza (flu) or allergies, particularly in the case of persistent or recurring symptoms.

2. Recommended actions:
   - Mild cold: Rest, stay hydrated, get plenty of sleep, and over-the-counter cold remedies can provide symptomatic relief. It's important to blow your nose gently.
   - Severe cold or flu: If you have a high fever, persistent cough, or severe chest pain, shortness of breath, or severe headache, you should consult a healthcare provider as these could indicate more serious conditions.
   - Allergies: Allergy symptoms can be managed with over-the-counter allergy medications, but a consultation with a healthcare provider may be necessary to confirm the cause and determine the best treatment.

3. Precautionary measures:
   - Wash hand